**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [1]:
import pandas as pd

In [57]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [58]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [59]:
for k,v in geo_data.items():
    for i in range(len(v)):
        keywords.loc[ keywords['keyword'].str.contains(v[i]) == True, 'region'] = k
        keywords.loc[ keywords['keyword'].str.contains(v[i]) == False, 'region'] = 'undefined'
#         print(keywords[(keywords['keyword'].str.contains(v[i]) == True)])
print(keywords)


                                          keyword     shows     region
0                                              вк  64292779  undefined
1                                   одноклассники  63810309  undefined
2                                           порно  41747114  undefined
3                                            ютуб  39995567  undefined
4                                       вконтакте  21014195  undefined
5                      одноклассники моя страница  19976138  undefined
6                                            майл  15144673  undefined
7                                           авито  14786593  undefined
8                                      переводчик  14453469  undefined
9                                          яндекс  13344734  undefined
10                                сбербанк онлайн  10725673  undefined
11                                           mail   7687204  undefined
12                                           игры   6075762  undefined
13    

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [59]:
ratings = pd.read_csv('ratings_2.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [113]:
def class_rating (table, low_r = 2, middle_r = 4, high_r= [4.5, 5] ):
    ratings.loc[ ratings['rating'] <= low_r, 'class'] = 'низкий рейтинг'
    ratings.loc[ (ratings['rating'] <= middle_r) & (ratings['rating'] > low_r), 'class'] = 'средний рейтинг'
    ratings.loc[ (ratings['rating'] == high_r[0] ) | (ratings['rating'] == high_r[1]), 'class'] = 'высокий рейтинг'
    return table

In [114]:
class_rating(ratings).head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [34]:
# from datetime import datetime

In [57]:
ratings_f = ratings.groupby('userId').count()['movieId']
ratings_f[ratings_f>100].index
# ratings_f['userId'].tolist()
filter_user =ratings[ratings['userId'].isin(ratings_f)]
max_min_time = filter_user.groupby('userId').agg(['max','min'])['timestamp']
Lifetime = max_min_time['max'] - max_min_time['min']
round(Lifetime.mean()/86400, 2)

204.43

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [190]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [191]:
years=list(range(1950,2011))

In [192]:
"""вариант с функцией"""
def return_year(row):
    for x in years:
        if str(x) in row['title']:
            return str(x)
    return '1900'

In [198]:
movies['years'] = movies.apply(return_year, axis=1)
movies.tail()

,movieId,title,genres,years
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,1900
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi,1900
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,1900
9123,164977,The Gay Desperado (1936),Comedy,1900
9124,164979,"Women of '69, Unboxed",Documentary,1900


In [126]:
# """вариант без функции"""
# for x in years:
#     movies.loc[movies['title'].str.contains(str(x)), 'years'] = x
# movies['years'].fillna('1900', inplace=True)
# movies.head()

,movieId,title,genres,years
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [197]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.groupby('years').agg(['mean'])['rating'].sort_values('mean', ascending=False).head()

,mean
years,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
